In [1]:
import pandas as pd
import os
import numpy as np

# Parse through precipitation data & extract only dates in labeled range

In [2]:
directory = r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data"
df = pd.DataFrame()
for dirpath, dirnames, filenames in os.walk(directory):
    for file in filenames:
        if '_standard.csv' in file:
            abspath = dirpath + '\\' + file
            dfT = pd.read_csv(abspath, index_col='time', parse_dates=True)
            dfT = dfT.loc['2012':'2017']
            df = pd.concat([df, dfT])

C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


## One-Hot Encode Grid for Features

In [3]:
# Add column Grid to keep track of what grid the data is from
df['Grid'] = df['lat'].astype('str') + '|' + df['lon'].astype('str')
# One-Hot Encode Grid to use as feature later
df = pd.concat([df, pd.get_dummies(df['Grid'], prefix='GRID', drop_first=True)], axis=1)

## Define Method to Snap Lat/Long to Grid

In [4]:
def closest_grid(point, data):
    min_dist = 1_000_000
    for grid in data:
        dist = np.sum((point - grid)**2)
        if dist < min_dist:
            min_dist = dist
            best_grid = grid
    return best_grid[0], best_grid[1]

In [5]:
grid_data = []
for grid in np.unique(df['Grid']):
    lat, lon = grid.split('|')
    lat, lon = float(lat), float(lon)
    grid_data.append(np.array([lat, lon]))
point = np.array([45.5, -73.9])
closest_grid(point, grid_data)

(45.541666485, -73.875002685)

## One-Hot Encode Weekday & Month for Features

In [6]:
df['Weekday'] = df.index.weekday
df = pd.concat([df.drop(['Weekday'], axis=1), pd.get_dummies(df['Weekday'], prefix='WEEKDAY', drop_first=True)], axis=1)

df['Month'] = df.index.month
df = pd.concat([df.drop(['Month'], axis=1), pd.get_dummies(df['Month'], prefix='MONTH', drop_first=True)], axis=1)

## Set Index on Time/Lat/Lon

In [7]:
df = df.reset_index()
df.sort_values(['time', 'lat', 'lon']).set_index(['time','lat', 'lon'])

pr_CNRM-CM5_historical_rcp26_(kg m-2 d-1)  \
time                lat       lon                                                     
2012-01-01 12:00:00 45.458333 -73.958336                                   5.737305   
                              -73.875003                                   5.651855   
                              -73.791669                                   5.615234   
                              -73.708336                                   5.541992   
                              -73.625003                                   5.517578   
                              -73.541669                                   5.517578   
                    45.541666 -73.875003                                   5.822754   
                              -73.791669                                   5.712891   
                              -73.708336                                   5.700684   
                              -73.625003                                   5.676270   
                              -73.541669                                   5.603027   
                    45.625000 -73.791669                                   5.883789   
                              -73.708336                                   5.786133   
                              -73.625003                                   5.688477   
                              -73.541669                                   5.712891   
2012-01-02 12:00:00 45.458333 -73.958336                                   1.318298   
                              -73.875003                                   1.354919   
                              -73.791669                                   1.379333   
                              -73.708336                                   1.354919   
                              -73.625003                                   1.501404   
                              -73.541669                                   1.440369   
                    45.541666 -73.875003                                   1.257263   
                              -73.791669                                   1.293884   
                              -73.708336                                   1.391541   
                              -73.625003                                   1.525818   
                              -73.541669                                   1.464783   
                    45.625000 -73.791669                                   1.562439   
                              -73.708336                                   1.550232   
                              -73.625003                                   1.318298   
                              -73.541669                                   1.379333   
...                                                                             ...   
2017-12-30 12:00:00 45.458333 -73.958336                                   2.026306   
                              -73.875003                                   5.151367   
                              -73.791669                                   5.041504   
                              -73.708336                                   4.980469   
                              -73.625003                                   5.004883   
                              -73.541669                                   4.992676   
                    45.541666 -73.875003                                   2.197205   
                              -73.791669                                   5.285645   
                              -73.708336                                   5.322266   
                              -73.625003                                   5.297852   
                              -73.541669                                   5.126953   
                    45.625000 -73.791669                                   2.612244   
                              -73.708336                                   2.551208   
                              -73.625003

## Dump to CSV

In [8]:
df.drop(['Grid'], axis=1).to_csv(r"C:\Users\olive\Documents\GitHub\MilaClimateChangeHackathon2019\data\prepared_precipitation_data.csv")